In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Flatten, Dense, GaussianNoise, Dropout, RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.models import load_model
from tensorflow_addons.optimizers import MultiOptimizer

import os
import random
from joblib import load, dump
import gc
import sys

tf.keras.backend.set_floatx('float32')
input_shape = (128, 128, 3)

In [ ]:
base_model = MobileNetV2(include_top=False, input_shape=input_shape)

9420800/9406464 [==============================] - 1s 0us/step


In [ ]:
train_layers = ["Conv_1", "block_16_project", "block_16_depthwise", "block_16_expand",
                "block_15_project", "block_15_depthwise", "block_15_expand",
                "block_14_project", "block_14_depthwise", "block_14_expand"
                ]

for layer in base_model.layers:
    layer.trainable = layer.name in train_layers
    if layer.name in train_layers:
        print(layer.name)

block_14_expand
block_14_depthwise
block_14_project
block_15_expand
block_15_depthwise
block_15_project
block_16_expand
block_16_depthwise
block_16_project
Conv_1


In [ ]:
def mat_shuffle(matrixs):
    widths = np.cumsum([0]+[matrix.shape[1] for matrix in matrixs])
    big = np.hstack(matrixs)
    np.random.shuffle(big)
    return [big[:,widths[i]:widths[i+1]].copy() for i in range(len(matrixs))]

The labels of each face image is embedded in the file name, formated like [age]\_[gender]\_[race]\_[date&time].jpg

[age] is an integer from 0 to 116, indicating the age
[gender] is either 0 (male) or 1 (female)
[race] is an integer from 0 to 4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern).
[date&time] is in the format of yyyymmddHHMMSSFFF, showing the date and time an image was collected to UTKFace

In [ ]:
input_x = Input(shape=input_shape)

# augmentation layers
x = RandomFlip("horizontal")(input_x)
x = GaussianNoise(0.03)(x)
x = RandomRotation(0.3)(x)
x = RandomContrast(0.3)(x)

x = base_model(x)

x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
output_layer = (Dense(1, activation='tanh')(x)+1)/2

model = Model(input_x, output_layer)

base_layers = [layer for layer in base_model.layers if layer.name in train_layers]
added_layers = [model.layers[-1], model.layers[-3]]

optimizer = MultiOptimizer([(Adam(1e-5), base_layers), (Adam(1e-3), added_layers)])

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
results = dict()
c = 0
while True:
    for i in range(2):
        images = tf.image.resize(load(f"data/{i}_images.joblib"), (128,128))
        genders = load(f"data/{i}_genders.joblib")

        results[f"{c}_{i}"] = model.fit(images, genders[:, None], epochs=128, batch_size=256,
                              validation_split=0.2, use_multiprocessing=True, verbose=2)
        
        del images
        gc.collect()
        del genders
        gc.collect()
 
        
        model.save(f'models/{c}_model_gender.h5')
    c += 1

In [ ]:
def predict(url):
    image = tf.image.resize(tf.keras.utils.img_to_array(tf.keras.utils.load_img(url)), input_shape[:2])
    image = (image.numpy().astype('float32')-127.5)/127.5
    image = image[None, ...]
    return float(model.predict(image)*15.54)+35.3